In [1]:
import scipy.io
import numpy as np
import pickle
from os import listdir
from os.path import isfile, join
from PIL import Image

In [3]:
test_labels = scipy.io.loadmat('../../data/data_cars/cars_test_annos_labels.mat')
train_labels = scipy.io.loadmat('../../data/data_cars/cars_train_annos.mat')

train_data_path = "../../data/data_cars/cars_train/"
test_data_path = "../../data/data_cars/cars_test/"

In [4]:
test_labels.keys()

dict_keys(['__header__', '__version__', '__globals__', 'annotations'])

In [5]:
test_labels = np.array(test_labels.get('annotations'))
train_labels = np.array(train_labels.get('annotations'))

In [66]:
test_labels[0][0][4][0][0]

181

In [60]:
train_labels[0][1][4][0][0]

3

### Get test and train labels

In [7]:
# Length of test and train sets
len_test = len(test_labels[0])
len_train = len(train_labels[0])

y_test = np.empty(len_test, dtype="int16")
y_train = np.empty(len_train, dtype="int16")

# Test labels
for i in range(len_test):
    y_test[i] = test_labels[0][i][4][0][0]

# Train labels
for i in range(len_train):
    y_train[i] = train_labels[0][i][4][0][0]

In [8]:
y_test.shape
y_train.shape

(8144,)

In [10]:
y_test[:10]
y_train[:10]

array([ 14,   3,  91, 134, 106, 123,  89,  96, 167,  58], dtype=int16)

### Load in images as numpy array

In [15]:
# coding: utf-8

# Loading in Stanford Cars Dataset data

import scipy.io
import numpy as np
import pickle
from os import listdir
from os.path import isfile, join
from PIL import Image

# Paths to files
TEST_LABELS_PATH = '../../data/data_cars/cars_test_annos_labels.mat'
TRAIN_LABELS_PATH = '../../data/data_cars/cars_train_annos.mat'
TRAIN_DATA_PATH = "../../data/data_cars/cars_train/"  # Folder full of images
TEST_DATA_PATH = "../../data/data_cars/cars_test/"


def load_img(path, new_size = 256):
    """
    Loads in an images, so its sorter side will match to new side
    
    params:
        path: (string) location to the image
        new_size: (int) the new size of the image
    """
    im = Image.open(path)
    if im.size[0] < im.size[1]:
        size_perc = new_size/im.size[0]
    else:
        size_perc = new_size/im.size[1]
        
    size = (int(round(im.size[0]*size_perc, 0)), int(round(im.size[1]*size_perc, 0))) # New size of the image

    im = im.resize(size, Image.ANTIALIAS)
    rgb_im = im.convert('RGB')  # Some images are in Grayscale
    return np.array(rgb_im, dtype="float32")

# Get center of image array
def center_crop(img_mat, size = (224, 224)):
    w,h,c = img_mat.shape
    start_h = h//2-(size[1]//2)  # Size[1] - h of cropped image
    start_w = w//2-(size[0]//2)  # Size[0] - w of cropepd image
    return img_mat[start_w:start_w+size[0],start_h:start_h+size[1], :]


def load_data_cars(size = 256, size_crop = (224, 224)):

    # Path to data - change according your paths
    test_labels = scipy.io.loadmat(TEST_LABELS_PATH)  # Labels saved as matlab mat-s
    train_labels = scipy.io.loadmat(TRAIN_LABELS_PATH)


    # Get labels from Matlab matrix
    test_labels = np.array(test_labels.get('annotations'))
    train_labels = np.array(train_labels.get('annotations'))


    # ### Get test and train labels

    # Length of test and train sets
    len_test = len(test_labels[0])
    len_train = len(train_labels[0])

    y_test = np.empty(len_test, dtype="int16")
    y_train = np.empty(len_train, dtype="int16")

    # Test labels
    for i in range(len_test):
        y_test[i] = test_labels[0][i][4][0][0]  # Get labels out of annotations

    # Train labels
    for i in range(len_train):
        y_train[i] = train_labels[0][i][4][0][0]

    #
    print("Current minimal label", min(y_test), "and max:", max(y_test)) 
    # Should be 1, but want it to be 0, so we could use 1-hot vector
    y_test -= 1  # min label zero, max 195
    y_train -= 1
    print("New minimal label:", min(y_test), "and max:", max(y_test))

        
    # ### Load in images as numpy array

    path = TRAIN_DATA_PATH
    train_imgs = [f for f in listdir(path) if isfile(join(path, f))]
    path2 = TEST_DATA_PATH
    test_imgs = [f for f in listdir(path2) if isfile(join(path2, f))]


    # Fill in x_train array with train data

    x_train = np.empty((14, size, size, 3), dtype="float32")

    for i, img_path in enumerate(train_imgs):
        img_mat = load_img(TRAIN_DATA_PATH + img_path, new_size = size)  # First load and rescale image
        x_train[i] = center_crop(img_mat, size = (size, size))  # Second center crop the scaled image

    # Fill in x_test array with test data
    x_test = np.empty((14, size_crop[0], size_crop[1], 3), dtype="float32")

    for i, img_path in enumerate(test_imgs):    
        img_mat = load_img(TEST_DATA_PATH + img_path, new_size = size)  # First scale to 256-by-x
        x_test[i] = center_crop(img_mat, size = size_crop)  # Crop center of the image


    return ((x_train, y_train), (x_test, y_test))



In [16]:
(x_train, y_train), (x_test, y_test) = load_data_cars(256, (224,224))

Current minimal label 1 and max: 196
New minimal label: 0 and max: 195


In [18]:
x_train.shape

(14, 256, 256, 3)

In [19]:
x_test.shape

(14, 224, 224, 3)

## Test the model

In [39]:
import math, json, os, sys

import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing import image
import pickle
from sklearn.model_selection import train_test_split
from load_data_cars import load_data_cars
from image_gen_extended import ImageDataGenerator, random_crop
from keras.layers import GlobalAveragePooling2D, Dense, Flatten, AveragePooling2D
from keras.models import Model

SIZE_IMG = 256
SIZE_CROP = (224, 224)
BATCH_SIZE = 64
NR_CLASSES = 196  # Classes for cars
EPOCHS = 250
SEED = 333  # Random seed for reproducibility
MEAN = [103.939, 116.779, 123.68]
weights_file = "../../models/resnet50_cars.h5"

# Train 8144 and test 8041

In [21]:
#  If you are freezing initial layers, you should use imagenet mean/std. (https://discuss.pytorch.org/t/confused-about-the-image-preprocessing-in-classification/3965)            
x_train = x_train[..., ::-1]
x_test = x_test[..., ::-1]

for i in range(3):
    x_train[:,:,:,i] -= MEAN[i]
    x_test[:,:,:,i] -= MEAN[i]

In [40]:
    
print("Load model")
base_model = keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet', 
                                                  input_tensor=None, input_shape=(224,224,3), 
                                                  pooling=None, classes=NR_CLASSES)

# Atm all layers trainable -> Test with only base layers untrainable 
#for layer in base_model.layers:
#   layer.trainable=False

x_fc = base_model.output
#x_fc = AveragePooling2D((7, 7), name='avg_pool')(x)
x_fc = Flatten()(x_fc)
predictions = Dense(NR_CLASSES, activation='softmax')(x_fc)

model = Model(inputs=base_model.input, outputs=predictions)
model.load_weights(weights_file)
print(model.summary())


sgd = SGD(lr=0.0005, decay = 1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

Load model
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        input_2[0][0]                    
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
____________________________________________________________________________________________________
activation_50 (Activation)       (None, 112, 112, 64)  0           bn_conv1[0][0]                   
________________________________________________________________________________

____________________________________________________________________________________________________
activation_73 (Activation)       (None, 14, 14, 256)   0           bn4a_branch2b[0][0]              
____________________________________________________________________________________________________
res4a_branch2c (Conv2D)          (None, 14, 14, 1024)  263168      activation_73[0][0]              
____________________________________________________________________________________________________
res4a_branch1 (Conv2D)           (None, 14, 14, 1024)  525312      activation_71[0][0]              
____________________________________________________________________________________________________
bn4a_branch2c (BatchNormalizatio (None, 14, 14, 1024)  4096        res4a_branch2c[0][0]             
____________________________________________________________________________________________________
bn4a_branch1 (BatchNormalization (None, 14, 14, 1024)  4096        res4a_branch1[0][0]     

In [41]:
y_probs = model.predict(x_test, verbose=1)

14/14 [==============================] - 11s


In [43]:
y_preds = np.argmax(y_probs, axis=1)

In [44]:
y_preds

array([ 79, 165, 167, 187,  39, 116,  43,  41,  13, 127, 103,  89,  60, 163], dtype=int64)

In [42]:
y_test[:14]

array([180, 102, 144, 186, 184,  77, 117, 164,  31,  59,  48, 107, 115, 134], dtype=int16)

In [27]:
y_test_cat = keras.utils.to_categorical(y_test[:14], NR_CLASSES)
loss, accuracy = model.evaluate(x_test, y_test_cat, verbose=1)
print("Test: accuracy1 = %f  ;  loss1 = %f" % (accuracy, loss))

14/14 [==============================] - 11s
Test: accuracy1 = 0.000000  ;  loss1 = 5.410984
